In [1]:
#library

In [34]:
import requests
from bs4 import BeautifulSoup
import re
import os
import csv

In [3]:
#get data

In [4]:
url='https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number'
r=requests.get(url)
r.encoding = 'utf-8'

In [5]:
r.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of Pokémon by National Pokédex number - Bulbapedia, the community-driven Pokémon encyclopedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Pokémon_by_National_Pokédex_number","wgTitle":"List of Pokémon by National Pokédex number","wgCurRevisionId":3123839,"wgRevisionId":3123839,"wgArticleId":65356,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Lists of Pokémon","Lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateF

In [6]:
#compile data
soup=BeautifulSoup(r.text,"html5lib")

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Pokémon by National Pokédex number - Bulbapedia, the community-driven Pokémon encyclopedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Pokémon_by_National_Pokédex_number","wgTitle":"List of Pokémon by National Pokédex number","wgCurRevisionId":3123839,"wgRevisionId":3123839,"wgArticleId":65356,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Lists of Pokémon","Lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["","

In [10]:
#create folder

In [19]:
folder = os.path.exists('Pokemon_data')
if not folder:
#如果不存在，則建立新目錄
    os.makedirs('Pokemon_data')

In [20]:
#create file

In [64]:
f = open("Pokemon_data/" + 'pokemon_data.csv', "a+"
                     #open for writing, appending to the end of the file if it exists
                     #open for updating (reading and writing)
                     #https://docs.python.org/3/library/functions.html
                     , encoding='utf-8')

In [65]:
#open file and create column names
with open('pokemon_data.csv', 'a', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Kdex','Ndex','Name','Type1','Type2','Link',
               'HP','ATK','DEF','SP_ATK','SP_DEF',
               'Damaged Normally by','Weak to','Immue to','Resisant to'])

In [66]:
#arrange data
counter=0#counter
for i in soup.find('div',{'id':'mw-content-text'}).find_all('tr',{"style":"background:#FFF"}):
    counter=counter+1
    print('Loading...')
    pokemon_data=[]    
    #generate_number
    generate_number=i.find_all('td')[0].text.replace('\n',"")
    print('Kdex: ',generate_number)
    pokemon_data.append(generate_number)
    
    #national_number
    national_number=i.find_all('td')[1].text.replace('\n',"")
    print('Ndex: ',national_number)
    pokemon_data.append(national_number)
    
    #name
    name=i.find('a')['title']
    print('Name: ',name)
    pokemon_data.append(name)
    #type1
    type1=i.find_all('span')[0].text
    print('Type1: ',type1)
    pokemon_data.append(type1)
    
    #type2 if exist
    if len(i.find_all('span'))==2:
        type2=i.find_all('span')[1].text
        print('Type2: ',type2)
        pokemon_data.append(type2)
    else:
        type2='Null'
        print('Type2: ',type2)
        pokemon_data.append(type2)
    #link to specific attr
    link='https://bulbapedia.bulbagarden.net/'+i.find_all('a',
                                                          href=re.compile("^(/wiki/)((?!;)\S)*$"))[1]['href']
    print('Link: ',link)
    pokemon_data.append(link)
    
    
    #get specific attr for a pokemon
    url_pkm=link
    r_pkm=requests.get(url_pkm)
    r_pkm.encoding = 'utf-8'
    soup_pkm=BeautifulSoup(r_pkm.text,"html5lib")
    
    #hp
    hp=soup_pkm.find('tr',{'style':"background: #FF5959; text-align:center"}).find('div',{'style':'float:right'}).text
    print("HP: ",hp) 
    pokemon_data.append(hp)
    #atk
    atk=soup_pkm.find('tr',{'style':"background: #F5AC78; text-align:center"}).find('div',{'style':'float:right'}).text
    print("ATK: ",atk) 
    pokemon_data.append(atk)
    #def
    defense=soup_pkm.find('tr',{'style':"background: #FAE078; text-align:center"}).find('div',{'style':'float:right'}).text
    print("DEF: ",defense)
    pokemon_data.append(defense)
    #sp_atk
    sp_atk=soup_pkm.find('tr',{'style':"background: #9DB7F5; text-align:center"}).find('div',{'style':'float:right'}).text
    print("SP_ATK: ",sp_atk)  
    pokemon_data.append(sp_atk)
    #sp_def
    sp_defense=soup_pkm.find('tr',{'style':"background: #A7DB8D; text-align:center"}).find('div',{'style':'float:right'}).text
    print("SP_DEF: ",sp_defense)
    pokemon_data.append(sp_defense)
    #spd
    spd=soup_pkm.find('tr',{'style':"background: #FA92B2; text-align:center"}).find('div',{'style':'float:right'}).text
    print("SPD: ",spd)
    pokemon_data.append(spd)

    #type_effectiveness
    type_effectiveness=soup_pkm.find('table',{'class':'roundy','width':'100%','align':'center'}) 
    
    #get four dimentions 
    normal=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[1]
    weak=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[2]
    immue=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[3]
    resisant=type_effectiveness.find_all('table',{'class':'roundy','width':'100%','align':'center'})[4]
    #create list for each dimention
    normal_lst=[]
    weak_lst=[]
    immue_lst=[]
    resisant_lst=[]
    
    #get data into each list and print
    #Damaged Normally by
    for j in normal.find_all('span',{'style':'display:inline-block;'}):
        normal_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
    print('Damaged Normally by :',normal_lst)
    pokemon_data.append(str(normal_lst))
    
    #Weak to
    for j in weak.find_all('span',{'style':'display:inline-block;'}):
        weak_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
    print('Weak to :',weak_lst)
    pokemon_data.append(str(weak_lst))
    
    #Immue to
    for j in immue.find_all('span',{'style':'display:inline-block;'}):
        immue_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
    print('Immue to :',immue_lst)
    pokemon_data.append(str(immue_lst))
    
    #Resisant to
    for j in resisant.find_all('span',{'style':'display:inline-block;'}):
        resisant_lst.append(j.text.replace('\n','').strip().replace(' ','_'))
    print('Resisant to :',resisant_lst)
    pokemon_data.append(str(resisant_lst))
    
#writting data    
    
    with open('pokemon_data.csv', 'a', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(pokemon_data)
    
    print('Writting...')
    print('Done!!!')
    print('==============================================================================\n')
#counter for testing     
    #if counter ==10:
    #    break
    #else:
    #    continue
    

Loading...
Kdex:   #001
Ndex:   #001
Name:  Bulbasaur
Type1:  Grass
Type2:  Poison
Link:  https://bulbapedia.bulbagarden.net//wiki/Bulbasaur_(Pok%C3%A9mon)
HP:  45
ATK:  49
DEF:  49
SP_ATK:  65
SP_DEF:  65
SPD:  45
Damaged Normally by : ['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_1×', 'Bug_1×', 'Ghost_1×', 'Steel_1×', 'Dragon_1×', 'Dark_1×']
Weak to : ['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']
Immue to : ['None']
Resisant to : ['Fighting_½×', 'Water_½×', 'Grass_¼×', 'Electric_½×', 'Fairy_½×']
Writting...
Done!!!

Loading...
Kdex:   #002
Ndex:   #002
Name:  Ivysaur
Type1:  Grass
Type2:  Poison
Link:  https://bulbapedia.bulbagarden.net//wiki/Ivysaur_(Pok%C3%A9mon)
HP:  60
ATK:  62
DEF:  63
SP_ATK:  80
SP_DEF:  80
SPD:  60
Damaged Normally by : ['Normal_1×', 'Poison_1×', 'Ground_1×', 'Rock_1×', 'Bug_1×', 'Ghost_1×', 'Steel_1×', 'Dragon_1×', 'Dark_1×']
Weak to : ['Flying_2×', 'Fire_2×', 'Psychic_2×', 'Ice_2×']
Immue to : ['None']
Resisant to : ['Fighting_½×', 'Water_½×', 'Grass_¼×'

HP:  65
ATK:  80
DEF:  40
SP_ATK:  45
SP_DEF:  80
SPD:  75
Damaged Normally by : ['Normal_1×', 'Ground_1×', 'Ghost_1×', 'Steel_1×', 'Water_1×', 'Electric_1×', 'Ice_1×', 'Dragon_1×', 'Dark_1×']
Weak to : ['Flying_2×', 'Rock_2×', 'Fire_2×', 'Psychic_2×']
Immue to : ['None']
Resisant to : ['Fighting_¼×', 'Poison_½×', 'Bug_½×', 'Grass_¼×', 'Fairy_½×']
Writting...
Done!!!

Loading...
Kdex:   #016
Ndex:   #016
Name:  Pidgey
Type1:  Normal
Type2:  Flying
Link:  https://bulbapedia.bulbagarden.net//wiki/Pidgey_(Pok%C3%A9mon)
HP:  40
ATK:  45
DEF:  40
SP_ATK:  35
SP_DEF:  35
SPD:  56
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poison_1×', 'Steel_1×', 'Fire_1×', 'Water_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Rock_2×', 'Electric_2×', 'Ice_2×']
Immue to : ['Ground_0×', 'Ghost_0×']
Resisant to : ['Bug_½×', 'Grass_½×']
Writting...
Done!!!

Loading...
Kdex:   #017
Ndex:   #017
Name:  Pidgeotto
Type1:  Normal
Type2:  Flying
Link:  https://bulbapedia.bulba

HP:  60
ATK:  90
DEF:  55
SP_ATK:  90
SP_DEF:  80
SPD:  100
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Poison_1×', 'Rock_1×', 'Bug_1×', 'Ghost_1×', 'Fire_1×', 'Water_1×', 'Grass_1×', 'Psychic_1×', 'Ice_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Ground_2×']
Immue to : ['None']
Resisant to : ['Flying_½×', 'Steel_½×', 'Electric_½×']
Writting...
Done!!!

Loading...
Kdex:   #027
Ndex:   #027
Name:  Sandshrew
Type1:  Ground
Type2:  Null
Link:  https://bulbapedia.bulbagarden.net//wiki/Sandshrew_(Pok%C3%A9mon)
HP:  50
ATK:  75
DEF:  85
SP_ATK:  20
SP_DEF:  30
SPD:  40
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Ground_1×', 'Bug_1×', 'Ghost_1×', 'Steel_1×', 'Fire_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Water_2×', 'Grass_2×', 'Ice_2×']
Immue to : ['Electric_0×']
Resisant to : ['Poison_½×', 'Rock_½×']
Writting...
Done!!!

Loading...
Kdex:    
Ndex:   #027
Name:  Sandshrew
Type1:  Ice
Type2:  Steel
Link:  https://bulbapedia.bulbaga

HP:  38
ATK:  41
DEF:  40
SP_ATK:  50
SP_DEF:  65
SPD:  65
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poison_1×', 'Ghost_1×', 'Electric_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×']
Weak to : ['Ground_2×', 'Rock_2×', 'Water_2×']
Immue to : ['None']
Resisant to : ['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', 'Ice_½×', 'Fairy_½×']
Writting...
Done!!!

Loading...
Kdex:   #038
Ndex:   #038
Name:  Ninetales
Type1:  Fire
Type2:  Null
Link:  https://bulbapedia.bulbagarden.net//wiki/Ninetales_(Pok%C3%A9mon)
HP:  73
ATK:  76
DEF:  75
SP_ATK:  81
SP_DEF:  100
SPD:  100
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poison_1×', 'Ghost_1×', 'Electric_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×']
Weak to : ['Ground_2×', 'Rock_2×', 'Water_2×']
Immue to : ['None']
Resisant to : ['Bug_½×', 'Steel_½×', 'Fire_½×', 'Grass_½×', 'Ice_½×', 'Fairy_½×']
Writting...
Done!!!

Loading...
Kdex:    
Ndex:   #038
Name:  Ninetales
Type1:  Ice
Type2:  Fairy
Link:  https://bulbapedia.

HP:  10
ATK:  55
DEF:  25
SP_ATK:  35
SP_DEF:  45
SPD:  95
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Ground_1×', 'Bug_1×', 'Ghost_1×', 'Steel_1×', 'Fire_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Water_2×', 'Grass_2×', 'Ice_2×']
Immue to : ['Electric_0×']
Resisant to : ['Poison_½×', 'Rock_½×']
Writting...
Done!!!

Loading...
Kdex:    
Ndex:   #050
Name:  Diglett
Type1:  Ground
Type2:  Steel
Link:  https://bulbapedia.bulbagarden.net//wiki/Diglett_(Pok%C3%A9mon)
HP:  10
ATK:  55
DEF:  25
SP_ATK:  35
SP_DEF:  45
SPD:  95
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Ground_1×', 'Bug_1×', 'Ghost_1×', 'Steel_1×', 'Fire_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Water_2×', 'Grass_2×', 'Ice_2×']
Immue to : ['Electric_0×']
Resisant to : ['Poison_½×', 'Rock_½×']
Writting...
Done!!!

Loading...
Kdex:   #051
Ndex:   #051
Name:  Dugtrio
Type1:  Ground
Type2:  Null
Link:  https://bulbapedia.bulbagarden.net//wi

...

HP:  61
ATK:  123
DEF:  60
SP_ATK:  60
SP_DEF:  50
SPD:  136
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poison_1×', 'Ground_1×', 'Rock_1×', 'Bug_1×', 'Ghost_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Grass_2×', 'Electric_2×']
Immue to : ['None']
Resisant to : ['Steel_½×', 'Fire_½×', 'Water_½×', 'Ice_½×']
Writting...
Done!!!

Loading...
Kdex:   #310
Ndex:   #848
Name:  Toxel
Type1:  Electric
Type2:  Poison
Link:  https://bulbapedia.bulbagarden.net//wiki/Toxel_(Pok%C3%A9mon)
HP:  40
ATK:  38
DEF:  35
SP_ATK:  54
SP_DEF:  35
SPD:  40
Damaged Normally by : ['Normal_1×', 'Rock_1×', 'Ghost_1×', 'Fire_1×', 'Water_1×', 'Ice_1×', 'Dragon_1×', 'Dark_1×']
Weak to : ['Ground_4×', 'Psychic_2×']
Immue to : ['None']
Resisant to : ['Fighting_½×', 'Flying_½×', 'Poison_½×', 'Bug_½×', 'Steel_½×', 'Grass_½×', 'Electric_½×', 'Fairy_½×']
Writting...
Done!!!

Loading...
Kdex:   #311
Ndex:   #849
Name:  Toxtricity
Type1:  Electric
Type2:  Poison
Link:  https://bul

HP:  95
ATK:  120
DEF:  65
SP_ATK:  95
SP_DEF:  75
SPD:  60
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Ground_1×', 'Rock_1×', 'Bug_1×', 'Fire_1×', 'Water_1×', 'Grass_1×', 'Electric_1×', 'Ice_1×']
Weak to : ['Poison_2×', 'Steel_2×', 'Fairy_2×']
Immue to : ['Psychic_0×', 'Dragon_0×']
Resisant to : ['Ghost_½×', 'Dark_¼×']
Writting...
Done!!!

Loading...
Kdex:   #033
Ndex:   #862
Name:  Obstagoon
Type1:  Dark
Type2:  Normal
Link:  https://bulbapedia.bulbagarden.net//wiki/Obstagoon_(Pok%C3%A9mon)
HP:  93
ATK:  90
DEF:  101
SP_ATK:  60
SP_DEF:  81
SPD:  95
Damaged Normally by : ['Normal_1×', 'Flying_1×', 'Poison_1×', 'Ground_1×', 'Rock_1×', 'Steel_1×', 'Fire_1×', 'Water_1×', 'Grass_1×', 'Electric_1×', 'Ice_1×', 'Dragon_1×']
Weak to : ['Fighting_4×', 'Bug_2×', 'Fairy_2×']
Immue to : ['Ghost_0×', 'Psychic_0×']
Resisant to : ['Dark_½×']
Writting...
Done!!!

Loading...
Kdex:   #183
Ndex:   #863
Name:  Perrserker
Type1:  Steel
Type2:  Null
Link:  https://bulbapedia.bulbagard

HP:  75
ATK:  80
DEF:  110
SP_ATK:  65
SP_DEF:  90
SPD:  50
Damaged Normally by : ['Normal_1×', 'Flying_1×', 'Poison_1×', 'Ground_1×', 'Bug_1×', 'Ghost_1×', 'Water_1×', 'Grass_1×', 'Electric_1×', 'Psychic_1×', 'Dragon_1×', 'Dark_1×', 'Fairy_1×']
Weak to : ['Fighting_2×', 'Rock_2×', 'Steel_2×', 'Fire_2×']
Immue to : ['None']
Resisant to : ['Ice_½×']
Writting...
Done!!!

Loading...
Kdex:   #337
Ndex:   #876
Name:  Indeedee
Type1:  Psychic
Type2:  Normal
Link:  https://bulbapedia.bulbagarden.net//wiki/Indeedee_(Pok%C3%A9mon)
HP:  60
ATK:  65
DEF:  55
SP_ATK:  105
SP_DEF:  95
SPD:  95
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Flying_1×', 'Poison_1×', 'Ground_1×', 'Rock_1×', 'Steel_1×', 'Fire_1×', 'Water_1×', 'Grass_1×', 'Electric_1×', 'Ice_1×', 'Dragon_1×', 'Fairy_1×']
Weak to : ['Bug_2×', 'Dark_2×']
Immue to : ['Ghost_0×']
Resisant to : ['Psychic_½×']
Writting...
Done!!!

Loading...
Kdex:   #344
Ndex:   #877
Name:  Morpeko
Type1:  Electric
Type2:  Dark
Link:  https://bulbapedia.

HP:  92
ATK:  130
DEF:  115
SP_ATK:  80
SP_DEF:  115
SPD:  138
Damaged Normally by : ['Normal_1×', 'Flying_1×', 'Ground_1×', 'Rock_1×', 'Ghost_1×', 'Fire_1×', 'Water_1×', 'Grass_1×', 'Electric_1×', 'Psychic_1×', 'Ice_1×', 'Fairy_1×']
Weak to : ['Poison_2×', 'Steel_2×']
Immue to : ['Dragon_0×']
Resisant to : ['Fighting_½×', 'Bug_½×', 'Dark_½×']
Writting...
Done!!!

Loading...
Kdex:   #399
Ndex:   #889
Name:  Zamazenta
Type1:  Fighting
Type2:  Null
Link:  https://bulbapedia.bulbagarden.net//wiki/Zamazenta_(Pok%C3%A9mon)
HP:  92
ATK:  130
DEF:  115
SP_ATK:  80
SP_DEF:  115
SPD:  138
Damaged Normally by : ['Normal_1×', 'Fighting_1×', 'Poison_1×', 'Ground_1×', 'Ghost_1×', 'Steel_1×', 'Fire_1×', 'Water_1×', 'Grass_1×', 'Electric_1×', 'Ice_1×', 'Dragon_1×']
Weak to : ['Flying_2×', 'Psychic_2×', 'Fairy_2×']
Immue to : ['None']
Resisant to : ['Rock_½×', 'Bug_½×', 'Dark_½×']
Writting...
Done!!!

Loading...
Kdex:   #399
Ndex:   #889
Name:  Zamazenta
Type1:  Fighting
Type2:  Steel
Link:  https://b

AttributeError: 'NoneType' object has no attribute 'find'

In [67]:
f.close()